In [1]:
import pandas as pd
import numpy as np
import warnings
import warnings
from datetime import datetime
warnings.filterwarnings("ignore")

# func

In [123]:
def preprocess_tp_data(path):
  tp_data = pd.read_csv(path)
  tp_data.drop_duplicates(inplace = True)
  tp_data.last_r.loc[tp_data.last_r < 0] = -1
  tp_data.state_close_estimate.fillna(tp_data.last_r , inplace = True)
  tp_data.state_max_r.fillna(tp_data.last_r , inplace = True)
  tp_data.state_MA_r.fillna(tp_data.last_r, inplace = True)
  tp_data.state_RSI_r.fillna(tp_data.last_r, inplace = True)
  tp_data.state_MA_r.loc[tp_data.state_MA_r < -1] = -1
  tp_data.state_max_r.loc[tp_data.state_max_r < -1] = -1
  tp_data.state_chain.loc[~tp_data.state_chain.isna()] = tp_data.state_chain.loc[~tp_data.state_chain.isna()].apply(lambda x:str(x).strip('][').split(', '))
  tp_data.state_chain.loc[~tp_data.state_chain.isna()] = tp_data.state_chain.loc[~tp_data.state_chain.isna()].apply(lambda x:[float(element) for element in x])
  tp_data['state_day_start'] = tp_data['state_day_start'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
  tp_data['state_day_end'] = tp_data['state_day_end'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
  return tp_data
tp_data = preprocess_tp_data('/Users/tranthong/Desktop/stock-app/state_output_new.csv')

In [ ]:
def get_return_with_max_estimate(state_return_list, min_thresh = 5, cut_thresh = 0.7):
    c = np.array(state_return_list)
    x = c[c > min_thresh]
    if len(x) > 0:
        list_temp = [x[0]]
        for i in range(1, len(x)):
            temp = x[0:i].max() * cut_thresh
            list_temp.append(temp)
        temp_df = pd.DataFrame()
        temp_df['a'] = x
        temp_df['b'] = list_temp
        return temp_df.a.iloc[-1] if temp_df.loc[temp_df.a < temp_df.b].empty else  temp_df.loc[temp_df.a < temp_df.b].iloc[0].a
    else:
        lost_r = c[-1] if c[-1] > 0 else -1
        return lost_r

#concat_chain_data['estimate_with_thresh'] = concat_chain_data.state_chain.apply(lambda x : get_return_with_max_estimate(x) )

# TP data

In [92]:
tp_data = pd.read_csv('/Users/tranthong/Desktop/stock-app/state_output_new.csv')
tp_data.head(5)

,state_ticker,state_order,state_day_start,state_day_end,state_open,state_stop_loss,state_close_estimate,state_max_r,state_MA_r,state_RSI_r,last_r,state_chain
0,MRK,1,2022-11-09 14:55:00,2022-11-09 19:50:00,102.024,101.814,NaN,7.504762,0.885714,NaN,0.885714,"[1.5999999999999324, 3.1238095238094337, 3.552..."
1,MRK,1,2022-11-16 14:55:00,2022-11-16 15:00:00,99.946,99.656,NaN,-0.572414,NaN,NaN,-0.882759,"[-0.572413793103425, -0.882758620689637]"
2,MRK,2,2022-11-16 19:35:00,2022-11-16 19:55:00,99.898,100.028,NaN,0.984615,NaN,NaN,-1.000000,"[0.9846153846153123, 0.75384615384609, 0.52307..."
3,MRK,1,2022-11-25 15:30:00,2022-11-25 15:40:00,107.268,107.076,NaN,-0.458333,NaN,NaN,-0.979167,"[-0.770833333333284, -0.45833333333328397, -0...."
4,MRK,1,2022-11-29 18:25:00,2022-11-29 18:55:00,108.816,108.592,NaN,0.241071,NaN,NaN,-1.000000,"[0.10714285714285941, -0.42857142857143765, -0..."


In [101]:

tp_data = preprocess_tp_data('/Users/tranthong/Desktop/stock-app/state_output_new.csv')

tp_data[['state_close_estimate','state_max_r','state_MA_r','state_RSI_r','last_r']].describe()

,state_close_estimate,state_max_r,state_MA_r,state_RSI_r,last_r
count,47.000000,47.000000,47.000000,47.000000,47.000000
mean,0.840918,3.432424,0.707166,0.754433,0.705038
std,5.450067,7.524614,5.054256,5.363180,5.054968
min,-1.000000,-1.000000,-1.000000,-1.000000,-1.000000
25%,-1.000000,-0.054613,-1.000000,-1.000000,-1.000000
50%,-1.000000,0.643478,-1.000000,-1.000000,-1.000000
75%,-1.000000,3.200340,-1.000000,-1.000000,-1.000000
max,25.957143,34.671429,25.957143,33.528571,25.957143


In [98]:
tp_data[['state_close_estimate','state_max_r','state_MA_r','state_RSI_r','last_r']].sum()

state_close_estimate     39.523157
state_max_r             161.323927
state_MA_r               33.236787
state_RSI_r              35.458349
last_r                   33.136787
dtype: float64

In [102]:
tp_data.head(5)

,state_ticker,state_order,state_day_start,state_day_end,state_open,state_stop_loss,state_close_estimate,state_max_r,state_MA_r,state_RSI_r,last_r,state_chain
0,MRK,1,2022-11-09 14:55:00,2022-11-09 19:50:00,102.024,101.814,0.885714,7.504762,0.885714,0.885714,0.885714,"[1.5999999999999324, 3.1238095238094337, 3.552..."
1,MRK,1,2022-11-16 14:55:00,2022-11-16 15:00:00,99.946,99.656,-1.000000,-0.572414,-1.000000,-1.000000,-1.000000,"[-0.572413793103425, -0.882758620689637]"
2,MRK,2,2022-11-16 19:35:00,2022-11-16 19:55:00,99.898,100.028,-1.000000,0.984615,-1.000000,-1.000000,-1.000000,"[0.9846153846153123, 0.75384615384609, 0.52307..."
3,MRK,1,2022-11-25 15:30:00,2022-11-25 15:40:00,107.268,107.076,-1.000000,-0.458333,-1.000000,-1.000000,-1.000000,"[-0.770833333333284, -0.45833333333328397, -0...."
4,MRK,1,2022-11-29 18:25:00,2022-11-29 18:55:00,108.816,108.592,-1.000000,0.241071,-1.000000,-1.000000,-1.000000,"[0.10714285714285941, -0.42857142857143765, -0..."


# concat data

In [42]:
import glob , os
concat_data = pd.DataFrame()
for path in glob.glob("/Users/tranthong/Desktop/stock-app/state_output/state_output/*.csv"):
    tp_data = preprocess_tp_data(path)
    concat_data = pd.concat([tp_data, concat_data])
concat_data.reset_index(inplace=True)
concat_data.head(5)

,index,state_ticker,state_order,state_day_start,state_day_end,state_open,state_stop_loss,state_close_estimate,state_max_r,state_MA_r,state_RSI_r,last_r,state_chain
0,0,MRK,2,1.662560e+09,1.662561e+09,86.386,86.671,-1.000,-0.119298,-1.0,-1.000000,-1.0,"[0.11929824561405766, 0.4701754385964981, 0.82..."
1,1,MRK,2,1.662646e+09,1.662647e+09,86.508,86.838,-1.000,0.296970,-1.0,-1.000000,-1.0,"[-0.2969696969696954, -0.11515151515150576, 0...."
2,2,MRK,2,1.663012e+09,1.663163e+09,87.692,87.812,8.536,17.266667,3.1,8.016667,3.1,"[-0.09999999999989342, -8.016666666667222, -7...."
3,3,MRK,2,1.663250e+09,1.663252e+09,86.412,86.632,-1.000,0.509091,-1.0,-1.000000,-1.0,"[-0.5090909090909526, -0.5090909090909526, -0...."
4,4,MRK,1,1.663338e+09,1.663339e+09,87.218,86.998,-1.000,-0.218182,-1.0,-1.000000,-1.0,NaN


In [34]:
total_state = len(concat_data)
profit_state = len(concat_data.loc[concat_data.state_close_estimate > 0]) 
loss_state = total_state -  profit_state

total_profit = concat_data.state_close_estimate.sum()
total_win_profit = concat_data.state_close_estimate.loc[concat_data.state_close_estimate > 0].sum()
total_loss_profit = concat_data.state_close_estimate.loc[concat_data.state_close_estimate < 0].sum()
print(total_state , loss_state , profit_state , profit_state/total_state)
print(total_profit,total_loss_profit,total_win_profit)

1046 886 160 0.15296367112810708
325.74728183040054 -885.0 1210.7472818304004


In [35]:
concat_chain_data = concat_data.loc[(concat_data.state_chain.notna())]
concat_chain_data.state_close_estimate.loc[concat_chain_data.state_close_estimate>0].sum()

649.9320075821444

In [ ]:
concat_chain_data = concat_data.loc[(concat_data.state_chain.notna())]
concat_chain_data['len_state_chain']  = concat_chain_data.state_chain.apply(lambda x : len(x))
concat_chain_data = concat_chain_data.loc[concat_chain_data.len_state_chain > 2]
concat_chain_data.head(50)

In [264]:
concat_chain_data = concat_data.loc[(concat_data.state_chain.notna())]
concat_chain_data['len_state_chain']  = concat_chain_data.state_chain.apply(lambda x : len(x))
concat_chain_data.loc[(concat_chain_data.len_state_chain <= 2) & (concat_chain_data.state_close_estimate > 0)].state_close_estimate.sum() 

4.446544523588392

In [242]:
concat_chain_data.head(5)

,index,state_ticker,state_order,state_day_start,state_day_end,state_open,state_stop_loss,state_close_estimate,state_max_r,state_MA_r,state_RSI_r,last_r,state_chain,len_state_chain,estimate_with_thresh
0,0,MRK,2,1.662560e+09,1.662561e+09,86.386,86.671,-1.000,-0.119298,-1.0,-1.000000,-1.0,"[-0.11929824561405766, -0.4701754385964981, -0...",4,-1.000000
1,1,MRK,2,1.662646e+09,1.662647e+09,86.508,86.838,-1.000,0.296970,-1.0,-1.000000,-1.0,"[0.2969696969696954, 0.11515151515150576, -0.7...",4,-1.000000
2,2,MRK,2,1.663012e+09,1.663163e+09,87.692,87.812,8.536,17.266667,3.1,8.016667,3.1,"[0.09999999999989342, 8.016666666667222, 7.100...",77,17.266667
3,3,MRK,2,1.663250e+09,1.663252e+09,86.412,86.632,-1.000,0.509091,-1.0,-1.000000,-1.0,"[0.5090909090909526, 0.5090909090909526, 0.327...",9,-1.000000
7,7,MRK,2,1.663696e+09,1.663702e+09,85.906,86.016,-1.000,2.236364,-1.0,-1.000000,-1.0,"[0.32727272727274137, 1.3272727272727414, 2.14...",24,-1.000000


In [241]:
concat_chain_data['estimate_with_thresh'] = concat_chain_data.state_chain.apply(lambda x : get_return_with_max_estimate(x , 12 , 0.7) )
concat_chain_data.estimate_with_thresh.sum()

-27.32915049847987

# Stock_tp_data

In [104]:
tp_data = tp_data
stock_tp_data = tp_data[['state_ticker','state_close_estimate','state_max_r','state_MA_r','state_RSI_r','last_r']].groupby('state_ticker').sum()
stock_tp_data['win_count'] = tp_data[['state_ticker','state_close_estimate','state_max_r','state_MA_r','state_RSI_r','last_r']].loc[tp_data.state_MA_r > 0].groupby('state_ticker').count().state_MA_r
stock_tp_data['loss_count'] = tp_data[['state_ticker','state_close_estimate','state_max_r','state_MA_r','state_RSI_r','last_r']].loc[tp_data.state_MA_r < 0].groupby('state_ticker').count().state_MA_r
stock_tp_data.win_count.fillna(0 , inplace = True)
stock_tp_data['win_count'] = stock_tp_data['win_count'].astype(int) 
stock_tp_data.loc[stock_tp_data.state_close_estimate > -20].state_close_estimate.sum()

39.523156817196465

In [105]:
stock_tp_data

,state_close_estimate,state_max_r,state_MA_r,state_RSI_r,last_r,win_count,loss_count
state_ticker,,,,,,,
GILD,11.992923,31.199005,8.623077,-1.323077,8.523077,1,8
IBM,-2.742029,15.171245,-2.742029,6.463394,-2.742029,2,4
MMM,1.380000,10.113333,1.380000,4.400000,1.380000,1,1
MRK,22.842857,42.371130,22.842857,30.414286,22.842857,2,4
PFE,14.878429,39.402386,11.961905,-0.930952,11.961905,2,8
PYPL,-8.829023,23.066828,-8.829023,-3.565302,-8.829023,2,12


In [107]:
tp_data.head(5)

,state_ticker,state_order,state_day_start,state_day_end,state_open,state_stop_loss,state_close_estimate,state_max_r,state_MA_r,state_RSI_r,last_r,state_chain
0,MRK,1,2022-11-09 14:55:00,2022-11-09 19:50:00,102.024,101.814,0.885714,7.504762,0.885714,0.885714,0.885714,"[1.5999999999999324, 3.1238095238094337, 3.552..."
1,MRK,1,2022-11-16 14:55:00,2022-11-16 15:00:00,99.946,99.656,-1.000000,-0.572414,-1.000000,-1.000000,-1.000000,"[-0.572413793103425, -0.882758620689637]"
2,MRK,2,2022-11-16 19:35:00,2022-11-16 19:55:00,99.898,100.028,-1.000000,0.984615,-1.000000,-1.000000,-1.000000,"[0.9846153846153123, 0.75384615384609, 0.52307..."
3,MRK,1,2022-11-25 15:30:00,2022-11-25 15:40:00,107.268,107.076,-1.000000,-0.458333,-1.000000,-1.000000,-1.000000,"[-0.770833333333284, -0.45833333333328397, -0...."
4,MRK,1,2022-11-29 18:25:00,2022-11-29 18:55:00,108.816,108.592,-1.000000,0.241071,-1.000000,-1.000000,-1.000000,"[0.10714285714285941, -0.42857142857143765, -0..."


In [151]:
tp_data = preprocess_tp_data('/Users/tranthong/Desktop/stock-app/state_output_new.csv')
def get_return_with_max_estimate(state_return_list, min_thresh = 12, cut_thresh = 0.66):
    c = np.array(state_return_list)
    x = c[c > min_thresh]
    if len(x) > 0:
        list_temp = [x[0]]
        for i in range(1, len(x)):
            temp = x[0:i].max() * cut_thresh
            list_temp.append(temp)
        temp_df = pd.DataFrame()
        temp_df['a'] = x
        temp_df['b'] = list_temp
        return temp_df.a.iloc[-1] if temp_df.loc[temp_df.a < temp_df.b].empty else  temp_df.loc[temp_df.a < temp_df.b].iloc[0].a
    else:
        lost_r = c[-1] if c[-1] > 0 else -1
        return lost_r

tp_data['new_estimate'] = tp_data.state_chain.apply(lambda x: get_return_with_max_estimate(x, 12, 0.66))
tp_data.new_estimate.sum()

42.79521176225152

In [152]:
tp_data.state_close_estimate.sum()

39.52315681719646

In [ ]:
tp_data.loc[tp_data.state_close_estimate != tp_data.new_estimate].state_chain.iloc[0]

In [157]:
path =  '/Users/tranthong/Desktop/stock-app/stock_data_M5/'
def get_mt5_raw_data_range(ticker, start, end):
    stock_dir = path + ticker +'_2022.csv'
    rates =  pd.read_csv(stock_dir, index_col =[0])
    rates['index'] = rates['index'].apply(lambda x: datetime.strptime(x, '%Y-%m-%d %H:%M:%S'))
    a = rates.loc[ (rates['index'] > start) & (rates['index'] < end )]
    a.reset_index(inplace = True ,drop = True)
    a['day_num'] = a.index
    return a

start = tp_data.loc[tp_data.state_close_estimate != tp_data.new_estimate].state_day_start.iloc[0]
end = tp_data.loc[tp_data.state_close_estimate != tp_data.new_estimate].state_day_end.iloc[0]
ticker = tp_data.loc[tp_data.state_close_estimate != tp_data.new_estimate].state_ticker.iloc[0]
open = tp_data.loc[tp_data.state_close_estimate != tp_data.new_estimate].state_open.iloc[0]
stoploss = tp_data.loc[tp_data.state_close_estimate != tp_data.new_estimate].state_stop_loss.iloc[0]
r = abs(open -stoploss)
data =  get_mt5_raw_data_range(ticker, start, end)

In [158]:
(data.Close - open) / r

0       0.292308
1       0.523077
2      -0.015385
3       0.446154
4       0.984615
         ...    
297    18.215385
298    18.215385
299    18.215385
300    18.061538
301    17.138462
Name: Close, Length: 302, dtype: float64